# Bengali.AI Competition - Simple CNN Training (Ensemble)

### Team MuchLearningSuchWow

This notebook contains code for training the simple convolutional network we used in our ensemble. It is connected to Weights and Biases in order to keep track of progress and performance.

## Imports

In [ ]:
import os

from tqdm.auto import tqdm
import cv2
import pandas as pd
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
import psutil

from keras.layers import Conv2D, BatchNormalization, Activation, Add, MaxPool2D, Dense, \
    Dropout, GlobalAveragePooling2D, Concatenate, Input, Flatten, AveragePooling2D, Add
from keras import Model
from keras.regularizers import l2

import gc
import wandb
from wandb.keras import WandbCallback

## Filenames

In [ ]:
train_filename = 'input/bengaliai-cv19/train.csv'
model_filename - 'output/model_cnn.hdf5'

In [ ]:
if not os.path.isdir('output'):
    os.mkdir('output')

## Loading Dataframes

In [ ]:
train_df_ = pd.read_csv(train_filename)
train_df_ = train_df_.drop(['grapheme'], axis=1)

## Weights and Biases

In [ ]:
run = wandb.init(project='bengali')

In [ ]:
config = run.config
config.blocks = [
    {
        'width': 48,
        'output_width': 64,
        'cardinality': 4,
        'count': 4
    },
    {
        'width': 64,
        'output_width': 128,
        'cardinality': 4,
        'count': 4
    },
    {
        'width': 128,
        'output_width': 256,
        'cardinality': 4,
        'count': 4
    },
    {
        'width': 256,
        'output_width': 512,
        'cardinality': 4,
        'count': 4
    }
]
config.iChannels = 48
config.epochs = 120
config.max_lr = 0.0016
config.min_lr = 0.0004
config.n_cycles = 8
config.dropout = 0.3
config.batch_size = 200
config.validation_split = 0.08
config.steps_per_epoch = int(200840*(1-config.validation_split))//config.batch_size//4

## Building Simple CNN

In [ ]:
def build_cnn(blocks, iChannels, dropout, input_size=(64, 64, 1)):
    inputs = Input(shape=input_size)
    model = Conv2D(filters=iChannels, kernel_size=(7, 7), padding='SAME', activation='relu', input_shape=input_size)(inputs)
    for b in blocks:
        for i in range(b['count']):
            model = Conv2D(filters=b['width'], kernel_size=(3, 3), padding='SAME', activation='relu')(model)
        model = BatchNormalization(momentum=0.15)(model)
        model = MaxPool2D(pool_size=(2, 2))(model)
        model = Conv2D(filters=b['output_width'], kernel_size=(5, 5), padding='SAME', activation='relu')(model)
        model = Dropout(rate=dropout)(model)

    model = Flatten()(model)
    model = Dense(1024, activation="relu")(model)
    model = Dropout(rate=dropout)(model)
    dense = Dense(512, activation="relu")(model)

    head_root = Dense(168, activation='softmax', name='dense_a')(dense)
    head_vowel = Dense(11, activation='softmax', name='dense_b')(dense)
    head_consonant = Dense(7, activation='softmax', name='dense_c')(dense)

    model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])
    return model

In [ ]:
model = build_cnn(config.blocks, config.iChannels, config.dropout)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              loss_weights=[2, 1, 1], 
              metrics=['accuracy', keras.metrics.Recall()])

## Training

In [ ]:
validation_steps = int(200840*config.validation_split)//config.batch_size//4

In [ ]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)

        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [ ]:
learning_rate_reduction_root = ReduceLROnPlateau(monitor='dense_a_accuracy',
                                                 patience=3,
                                                 verbose=1,
                                                 factor=0.5,
                                                 min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='dense_b_accuracy',
                                                  patience=3,
                                                  verbose=1,
                                                  factor=0.5,
                                                  min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='dense_c_accuracy',
                                                      patience=3,
                                                      verbose=1,
                                                      factor=0.5,
                                                      min_lr=0.00001)

In [ ]:
datagen = MultiOutputDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range=0.15,  # Randomly zoom image
            width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False,   # randomly flip images
            rescale=1.0/255.0,
            validation_split=config.validation_split)

train_df_['image_id'] = train_df_['image_id'].astype(str)+'.png'

Y_train_root = pd.get_dummies(train_df_.set_index('image_id')['grapheme_root'], dtype=np.float32)
Y_train_vowel = pd.get_dummies(train_df_.set_index('image_id')['vowel_diacritic'], dtype=np.float32)
Y_train_consonant = pd.get_dummies(train_df_.set_index('image_id')['consonant_diacritic'], dtype=np.float32)

def generator_wrapper(gen: MultiOutputDataGenerator, df: pd.DataFrame, subset: str, batch_size: int):
    for flowx, flowy in gen.flow_from_dataframe(df, 
                                                color_mode='grayscale', 
                                                directory='data', 
                                                x_col='image_id',
                                                y_col='image_id', 
                                                class_mode='raw', 
                                                target_size=(64, 64), 
                                                subset=subset, 
                                                batch_size=batch_size, 
                                                shuffle=True):
        yield flowx, {
            'dense_a': Y_train_root.loc[flowy].values,
            'dense_b': Y_train_vowel.loc[flowy].values,
            'dense_c': Y_train_consonant.loc[flowy].values,
        }

validation_generator = generator_wrapper(datagen, train_df_, 'validation', config.batch_size)
model.fit_generator(generator_wrapper(datagen, train_df_, 'training', config.batch_size), 
                    validation_data=validation_generator, 
                    validation_steps=validation_steps,
                    epochs=config.epochs, 
                    steps_per_epoch=config.steps_per_epoch,
                    callbacks=[learning_rate_reduction_root, 
                               learning_rate_reduction_vowel, 
                               learning_rate_reduction_consonant,
                               WandbCallback(data_type='image')])

## Saving Model

In [ ]:
model.save(os.path.join(wandb.run.dir, model_filename))